---
sidebar_label: Google Gemini
keywords: [google gemini embeddings]
---

# Google 生成式 AI 嵌入 (AI Studio & Gemini API)

使用 `langchain-google-genai` 包中的 `GoogleGenerativeAIEmbeddings` 类连接到 Google 的生成式 AI 嵌入服务。

这将帮助您使用 LangChain 开始使用 Google 的生成式 AI 嵌入模型（如 Gemini）。有关 `GoogleGenerativeAIEmbeddings` 功能和配置选项的详细文档，请参阅 API 参考 [https://python.langchain.com/v0.2/api_reference/google_genai/embeddings/langchain_google_genai.embeddings.GoogleGenerativeAIEmbeddings.html](https://python.langchain.com/v0.2/api_reference/google_genai/embeddings/langchain_google_genai.embeddings.GoogleGenerativeAIEmbeddings.html)。

## 概述
### 集成详情

import { ItemTable } from "@theme/FeatureTables";

<ItemTable category="text_embedding" item="Google Gemini" />

## 设置

要访问 Google 生成式 AI 嵌入模型，您需要创建一个 Google Cloud 项目，启用 Generative Language API，获取 API 密钥，并安装 `langchain-google-genai` 集成包。

### 凭证

要使用 Google 生成式 AI 模型，您必须拥有一个 API 密钥。您可以在 Google AI Studio 中创建它。有关说明，请参阅 Google 文档 [https://ai.google.dev/gemini-api/docs/api-key](https://ai.google.dev/gemini-api/docs/api-key)。

获得密钥后，将其设置为环境变量 `GOOGLE_API_KEY`：

In [ ]:
import getpass
import os

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

为启用模型调用的自动跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

## 安装

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

## 用法

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[-0.024917153641581535,
 0.012005362659692764,
 -0.003886754624545574,
 -0.05774897709488869,
 0.0020742062479257584]

## 批量处理

你也可以一次性嵌入多个字符串以加快处理速度：

In [5]:
vectors = embeddings.embed_documents(
    [
        "Today is Monday",
        "Today is Tuesday",
        "Today is April Fools day",
    ]
)
len(vectors), len(vectors[0])

(3, 3072)

## 索引与检索

嵌入模型常用于检索增强生成（RAG）流程中，既可以作为索引数据的一部分，也可以用于后续的数据检索。更多详细说明，请参阅我们的 [RAG 教程](/docs/tutorials/)。

下方展示了如何使用我们上面初始化的 `embeddings` 对象来索引和检索数据。在本示例中，我们将在 `InMemoryVectorStore` 中索引和检索一个示例文档。

In [21]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

## 任务类型
`GoogleGenerativeAIEmbeddings` 可选支持 `task_type`，目前它必须是以下值之一：

- `SEMANTIC_SIMILARITY`: 用于生成经过优化的文本相似性评估的嵌入。
- `CLASSIFICATION`: 用于生成经过优化的、根据预设标签对文本进行分类的嵌入。
- `CLUSTERING`: 用于生成经过优化的、基于相似性对文本进行聚类的嵌入。
- `RETRIEVAL_DOCUMENT`, `RETRIEVAL_QUERY`, `QUESTION_ANSWERING`, and `FACT_VERIFICATION`: 用于生成经过优化的、用于文档搜索或信息检索的嵌入。
- `CODE_RETRIEVAL_QUERY`: 用于根据自然语言查询检索代码块，例如“排序数组”或“反转链表”。代码块的嵌入将使用 `RETRIEVAL_DOCUMENT` 计算。

默认情况下，我们在 `embed_documents` 方法中使用 `RETRIEVAL_DOCUMENT`，在 `embed_query` 方法中使用 `RETRIEVAL_QUERY`。如果您提供了任务类型，我们将将其用于所有方法。

In [ ]:
%pip install --upgrade --quiet  matplotlib scikit-learn

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

query_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07", task_type="RETRIEVAL_QUERY"
)
doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07", task_type="RETRIEVAL_DOCUMENT"
)

q_embed = query_embeddings.embed_query("What is the capital of France?")
d_embed = doc_embeddings.embed_documents(
    ["The capital of France is Paris.", "Philipp is likes to eat pizza."]
)

for i, d in enumerate(d_embed):
    print(f"Document {i+1}:")
    print(f"Cosine similarity with query: {cosine_similarity([q_embed], [d])[0][0]}")
    print("---")

Document 1
Cosine similarity with query: 0.7892893360164779
---
Document 2
Cosine similarity with query: 0.5438283285204146
---


## API 参考

有关 `GoogleGenerativeAIEmbeddings` 的详细文档说明和配置选项，请参阅 [API 参考](https://python.langchain.com/api_reference/google_genai/embeddings/langchain_google_genai.embeddings.GoogleGenerativeAIEmbeddings.html)。

## 其他配置

您可以将以下参数传递给 `ChatGoogleGenerativeAI` 以自定义 SDK 的行为：

- `client_options`: 要传递给 Google API Client 的 [Client Options](https://googleapis.dev/python/google-api-core/latest/client_options.html#module-google.api_core.client_options)，例如自定义的 `client_options["api_endpoint"]`
- `transport`: 要使用的传输方法，例如 `rest`、`grpc` 或 `grpc_asyncio`。